In [1]:
"""
This example trains on Split CIFAR10 with Naive strategy.
In this example each experience has a different task label.
We use a multi-head model with a separate classifier for each task.
"""

import argparse
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.models import SimpleMLP, as_multitask
from avalanche.training.supervised import Naive


def main(args):

    # model
    model = SimpleMLP(input_size=32 * 32 * 3, num_classes=10)
    model = as_multitask(model, "classifier")

    # CL Benchmark Creation
    benchmark = SplitCIFAR10(n_experiences=5, return_task_id=True)
    train_stream = benchmark.train_stream
    test_stream = benchmark.test_stream

    # Prepare for training & testing
    optimizer = Adam(model.parameters(), lr=0.01)
    criterion = CrossEntropyLoss()

    # Choose a CL strategy
    strategy = Naive(
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        train_mb_size=128,
        train_epochs=3,
        eval_mb_size=128,
        device=device,
    )

    # train and test loop
    for train_task in train_stream:
        strategy.train(train_task, num_workers=0)
        strategy.eval(test_stream)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--cuda",
        type=int,
        default=0,
        help="Select zero-indexed cuda device. -1 to use CPU.",
    )
    args = parser.parse_args()
    main(args)


usage: ipykernel_launcher.py [-h] [--cuda CUDA]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9014 --control=9012 --hb=9011 --Session.signature_scheme="hmac-sha256" --Session.key=b"7f7192da-c8f1-4c22-a9f8-bc03ada51756" --shell=9013 --transport="tcp" --iopub=9015 --f=c:\Users\Sasha\AppData\Roaming\jupyter\runtime\kernel-v2-2656ZiSZLbhkZwG5.json


SystemExit: 2

c:\Users\Sasha\Anaconda3\envs\avalanche\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from avalanche.benchmarks import SplitMNIST
# from avalanche.benchmarks.utils.data_loader import GroupBalancedDataLoader
from avalanche.benchmarks.utils.data_loader import TaskBalancedDataLoader
benchmark = SplitMNIST(5, return_task_id=True)

dl = TaskBalancedDataLoader([exp.dataset for exp in benchmark.train_stream], batch_size=4)
for x, y, t in dl:
    print(t.tolist())
    break

In [ ]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

# img = read_image("test/assets/encode_jpeg/grace_hopper_517x606.jpg")
img = read_image("grace_hopper_517x606.jpg")

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")